# Customer Churn Prediction - End-to-End ML Project
This notebook allows you to train a customer churn prediction model, serve it via a Flask API with ngrok, and test predictions.

In [ ]:
# Step 1 - Upload Dataset
from google.colab import files
import pandas as pd

uploaded = files.upload()  # Upload 'Telco-Customer-Churn.csv'
df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head()

In [ ]:

# Step 2 - Data Exploration
print(df.info())
print(df.describe())
print(df['Churn'].value_counts())


In [ ]:

# Step 3 - Data Preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)
df = pd.get_dummies(df, drop_first=True)

X = df.drop('Churn_Yes', axis=1)
y = df['Churn_Yes']

scaler = StandardScaler()
num_features = ['tenure','MonthlyCharges','TotalCharges']
X[num_features] = scaler.fit_transform(X[num_features])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:

# Step 4 - Train Model
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
import pickle

model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_pred))

# Save model
with open("churn_model.pkl", "wb") as f:
    pickle.dump(model, f)


In [ ]:

# Step 5 - Install Flask with ngrok
!pip install flask-ngrok


In [ ]:

# Step 6 - Flask API
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)

with open("churn_model.pkl", "rb") as f:
    model = pickle.load(f)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    features = pd.DataFrame([data])
    prediction = model.predict(features)
    probability = model.predict_proba(features)[:,1]
    return jsonify({'churn_prediction': int(prediction[0]), 'churn_probability': float(probability[0])})

app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-3:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.12/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
            

### Step 7 - Test API
Send a POST request to the ngrok URL generated above using Python `requests` or Postman.